In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [175]:
from scipy.io import loadmat
import numpy as np
from PIL import Image
import cv2
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import defaultdict
import torch.optim as optim
from torch.optim import lr_scheduler
import pandas as pd
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True
Image.MAX_IMAGE_PIXELS = None
Image_path='../input/backgroundremovedversion/*'
label_path='../input/mayo-clinic-strip-ai/train.csv'



In [96]:
# trans = transforms.Compose([
#     transforms.Resize([512,512]),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize([0.790595  , 0.67119867, 0.8091853 ], [0.05493367, 0.05985045, 0.04747279]) 
# ])

# class Dataset(Dataset):
#         def __init__(self, Image_path, label_path, transform):
#             df=pd.read_csv(label_path)
#             a=df['image_id'].tolist()
#             image_ls=[]
#             isin_list=[]
#             for i in glob.glob(Image_path):
#                 for j in glob.glob(f'{i}/*'):
#                     l=[]
#                     l.append(j)
#                     split=j.split('/')[-1]
#                     output=split.split('-')[0]
#                     l.append(output)
#                     image_ls.append(l)
#                     isin_list.append(output)
#             sorted(image_ls,key=lambda s: s[1])     
#             sorted(isin_list)
#             label_ls=[]
#             for i in isin_list:
#                 if i in a:
#                     label_ls.append(df[df['image_id']==i]['label'])
#             self.img_ls=image_ls
#             self.label_ls=label_ls
        
#         def __len__(self):
#             return len(self.img_ls)
#         def __getitem__(self, idx):
#             img_name = self.img_ls[idx]
#             img=Image.open(img_name[0])
#             img.load()
#             img = self.transform(img)
#             label = self.label_ls[idx]
#             return img,label

In [176]:
image_ls=[]
isin_list=[]
label_ls=[]
label_list=[]
for i in glob.glob('../input/backgroundremovedversion/*'):
    for j in glob.glob(f'{i}/*'):
        l=[]
        l.append(j)
        split=j.split('/')[-1]
        output=split.split('-')[0]
        l.append(output)
        image_ls.append(l)
        isin_list.append(output)
sorted(image_ls,key=lambda s: s[1])     
sorted(isin_list) 
for i in isin_list:
    if i in a:
        label_ls.extend(df[df['image_id']==i]['label'].tolist())
for i in label_ls:
    if i =="CE":
        label_list.append(0)
    else:
        label_list.append(1)   
trans = transforms.Compose([
    transforms.Resize([512,512]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.790595  , 0.67119867, 0.8091853 ], [0.05493367, 0.05985045, 0.04747279]) 
])
class Dataset(Dataset):
        def __init__(self, image_ls, label_ls,transform):
            self.img_ls=image_ls
            self.label_ls=label_ls
            self.transform=transform
        def __len__(self):
            return len(self.img_ls)
        def __getitem__(self, idx):
            img_name = self.img_ls[idx]
            img=Image.open(img_name[0])
            img.load()
            label = self.label_ls[idx]
            img = self.transform(img)
            return img,label

In [177]:

train_set = Dataset(image_ls,label_list,transform=trans)
train_loader = DataLoader(train_set, batch_size=8,num_workers=0,shuffle=True)
